In [180]:
import numpy as np
import pandas as pd


#아이템 기반 협업 필터링을 구현 일단은,,, 장소에 대한 피드가 쌓이기 전까지
# 컨텐츠 기반 필터링을 해도 좋겠지만,, 더미 데이터 다 넣어서 할꺼면 그냥 
#협업만 해도 상관 없지 않을까 싶다...

#시간 되면 하이브리드 하자

# 데이터 읽어 오기 

#장소 데이터 => locId추가 필요 /관광지 이름 /주소(내 위치 기반 대비)/ 워도 & 경도 /썸네일(지금은 없음)
#하은이가 보내준 데이터 다 가져와서 전처리해서 2개 컬럼으로 합치자 장소정보는 필요함
locs_1 = pd.read_csv("./전국관광지정보표준데이터.csv")
# locs_2 = pd.read_csv("./loc_datas/전국길관광정보표준데이터.csv",encoding='cp949')
locs_3 = pd.read_csv("./loc_datas/전국도시공원정보표준데이터.csv", encoding='cp949')
locs_4 = pd.read_csv("./loc_datas/전국야영(캠핑)장표준데이터.csv", encoding='cp949')
locs_5 = pd.read_csv("./loc_datas/전국지역특화거리표준데이터.csv", encoding='cp949')
locs_6 = pd.read_csv("./loc_datas/전국휴양림표준데이터.csv", encoding='cp949')

#유저 정보 데이터 671개 => 초기 회원가입으로 얻을 수 있는 데이터들
users = pd.read_csv("./user.csv", encoding="cp949")

#평점정보 데이터

# ratings = pd.read_csv("./ratings.csv")

users

,userSeq,userId,email,birth,gender,mbti,moods
0,1,u3Y00NaQze1ZG,test@naver.com,1995-05-27,True,infp,이국적인
1,2,utT9nWbWtdgaP,ssafy@naver.com,2003-05-06,True,estj,현대적인
2,3,uzYaePbx00003,ssafy3@naver.com,2002-12-12,True,intp,현대적인
3,4,uzYaePbx00004,ssafy4@naver.com,1996-07-17,True,estj,자연주의
4,5,uzYaePbx00005,ssafy5@naver.com,1995-05-05,True,estj,낭만적인
...,...,...,...,...,...,...,...
666,667,uzYaePbx00667,ssafy667@naver.com,2001-03-19,False,infj,자연주의
667,668,uzYaePbx00668,ssafy668@naver.com,1995-01-09,False,infp,평화로운
668,669,uzYaePbx00669,ssafy669@naver.com,2003-03-17,False,infj,복고풍의
669,670,uzYaePbx00670,ssafy670@naver.com,1996-08-23,False,estj,기타


In [197]:

# 관광지명, 소재지지번주소, 위도, 경도 => 데이터 가져오기
locs_1_filtered = locs_1[["관광지명", "소재지지번주소", "위도", "경도"]]
locs_3_filtered = locs_3[["공원명", "소재지지번주소", "위도", "경도"]]
locs_4_filtered = locs_4[["야영(캠핑)장명", "소재지지번주소", "위도", "경도"]]
locs_5_filtered = locs_5[["거리명", "소재지지번주소", "위도", "경도"]]
locs_6_filtered = locs_6[["휴양림명","소재지도로명주소","위도","경도"]]

#column명 변경하기
locs_1_filtered.rename(columns = {"관광지명":"name","소재지지번주소":"address", "위도":"latitude", "경도" : "longitude"}, inplace=True)
locs_3_filtered.rename(columns = {"공원명":"name","소재지지번주소":"address",  "위도":"latitude","경도" : "longitude"}, inplace=True)
locs_4_filtered.rename(columns = {"야영(캠핑)장명":"name", "소재지지번주소":"address",  "위도":"latitude","경도" : "longitude"}, inplace=True)
locs_5_filtered.rename(columns = {"거리명":"name", "소재지지번주소":"address",  "위도":"latitude","경도" : "longitude"}, inplace=True)
locs_6_filtered.rename(columns = {"휴양림명":"name","소재지도로명주소":"address",  "위도":"latitude","경도" : "longitude"}, inplace=True)


places = pd.concat([locs_1_filtered,locs_3_filtered, locs_4_filtered, locs_5_filtered, locs_6_filtered], ignore_index=True)

places = places.dropna(axis=0)

places = places.drop_duplicates(["name"], ignore_index=True)

places.index = places.index + 1

places["placeSeq"] = places.index

places = places[["placeSeq", "name", "address", "latitude", "longitude"]]

places = places[["placeSeq", "name"]]

places


,placeSeq,name
1,1,임진각관광지
2,2,공릉관광지
3,3,송정관광지
4,4,덕산온천관광단지
5,5,심우장
...,...,...
15201,15201,소백산자연휴양림
15202,15202,조령산자연휴양림
15203,15203,축령산
15204,15204,방장산자연휴양림


In [193]:
ratings = pd.read_csv("./ratings.csv")

ratings = ratings.drop_duplicates(["userSeq", "placeSeq"])


In [194]:
#협업 기반 필터링 -> 해보자..!


ratings = ratings.drop("timestamp", axis=1)

ratings

,userSeq,placeSeq,rating
0,1,2153,3
1,1,10810,3
2,1,5304,3
3,1,3802,2
4,1,6834,4
...,...,...,...
76109,671,8,4
76110,671,5959,4
76111,671,11919,5
76112,671,12457,3


In [203]:
# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['userSeq']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['userSeq'], x_test['placeSeq'])
    y_pred = np.array([model(user, place) for (user, place) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='userSeq', columns='placeSeq', values='rating')

rating_matrix

placeSeq,1,2,3,4,5,6,7,8,9,10,...,15195,15196,15197,15198,15199,15200,15201,15202,15203,15204
userSeq,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [205]:
# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

# 주어진 장소의 (placesSeq) 가중평균 rating을 계산하는 함수, 
# 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)
def CF_simple(userSeq, placeSeq):
    if placeSeq in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[userSeq].copy()
        # 현재 장소에 대한 모든 사용자의 rating값 가져오기
        place_ratings = rating_matrix[placeSeq].copy()
        # 현재 장소를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = place_ratings[place_ratings.isnull()].index
        # 현재 장소를 평가하지 않은 사용자의 rating (null) 제거
        place_ratings = place_ratings.dropna()
        # 현재 장소를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        # 현재 장소를 평가한 모든 사용자의 가중평균값 구하기
        mean_rating = np.dot(sim_scores, place_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating
print(CF_simple(186, 302))
# 정확도 계산
score(CF_simple)


4.0


C:\Users\SSAFY\AppData\Local\Temp/ipykernel_24388/1524237325.py:23: RuntimeWarning: invalid value encountered in double_scalars
  mean_rating = np.dot(sim_scores, place_ratings) / sim_scores.sum()


nan

In [210]:
# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['userSeq'], x_test['placeSeq'])
    y_pred = np.array([model(user, place, neighbor_size) for (user, place) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# Neighbor size를 정해서 예측치를 계산하는 함수 
def cf_knn(userSeq, placeSeq, neighbor_size=0):
    if placeSeq in rating_matrix:
        # 현재 장소와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[userSeq].copy()
        # 현재 장소에 대한 모든 사용자의 rating값 가져오기
        place_ratings = rating_matrix[placeSeq].copy()
        # 현재 장소를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = place_ratings[place_ratings.isnull()].index
        # 현재 장소를 평가하지 않은 사용자의 rating (null) 제거
        place_ratings = place_ratings.drop(none_rating_idx)
        # 현재 장소를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:          
            # 현재 장소를 평가한 모든 사용자의 가중평균값 구하기
            mean_rating = np.dot(sim_scores, place_ratings) / sim_scores.sum()
##### (3) Neighbor size가 지정된 경우
        else:                       
            # 해당 장소를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1: 
                # 지정된 neighbor size 값과 해당 장소를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                place_ratings = np.array(place_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도를 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                # 장소 rating을 neighbor size만큼 받기
                place_ratings = place_ratings[user_idx][-neighbor_size:]
                # 최종 예측값 계산 
                mean_rating = np.dot(sim_scores, place_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

print(cf_knn(186, 302))
# 정확도 계산
score(cf_knn, neighbor_size=30)

4.0


C:\Users\SSAFY\AppData\Local\Temp/ipykernel_24388/4032205509.py:41: RuntimeWarning: invalid value encountered in double_scalars
  mean_rating = np.dot(sim_scores, place_ratings) / sim_scores.sum()


nan

In [211]:
rating_matrix = ratings.pivot_table(values='rating', index='userSeq', columns='placeSeq')
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

print(rating_matrix)
# print(user_similarity)

placeSeq  1      2      3      4      5      6      7      8      9      \
userSeq                                                                   
1           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...         ...    ...    ...    ...    ...    ...    ...    ...    ...   
667         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
668         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
669         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
670         NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
671         NaN    NaN    NaN    NaN    NaN    NaN    NaN    4.0    NaN   

placeSeq  10     ...  15

In [218]:
def recom_place(userSeq, n_items, neighbor_size=30):
    # 현 사용자가 평가한 장소 가져오기
    user_place = rating_matrix.loc[userSeq].copy()
    for place in rating_matrix:
        # 현 사용자가 이미 평가한 장소는 제외 (평점을 0으로)        
        if pd.notnull(user_place.loc[place]):
            user_place.loc[place] = 0
        # 현 사용자가 평가하지 않은 장소의 예상 평점 계산
        else:
            user_place.loc[place] = cf_knn(userSeq, place, neighbor_size)
    # 장소를 예상 평점에 따라 정렬해서 제목을 뽑아서 돌려 줌
    place_sort = user_place.sort_values(ascending=False)[:n_items]
    recom_places = places.loc[place_sort.index]
    recommendations = recom_places['name']
    return recommendations

recom_place(userSeq=400, n_items=5, neighbor_size=30)

C:\Users\SSAFY\AppData\Local\Temp/ipykernel_24388/4032205509.py:41: RuntimeWarning: invalid value encountered in double_scalars
  mean_rating = np.dot(sim_scores, place_ratings) / sim_scores.sum()


placeSeq
10763                       소공원1(안)
13153                          마골공원
12284    LG Digital Park일반산업단지 근린공원
7510                          희망대공원
11659                        법원소공원3
Name: name, dtype: object